In [1]:
import torch


In [ ]:
# Discharge instructions for hadm 25142813

In [1]:
import pandas as pd
import re
import os
from pathlib import Path


# # data_dir = Path("..\\data\\scoring\\submission_01")
# refs_fullpath = r"C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\CODE\discharge_me\data\scoring\submission_01\submission.csv.csv"
# hyps_fullpath = r"C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\CODE\discharge_me\data\scoring\submission_01\discharge_target.csv"
refs_fullpath = 'submission.csv'

hyps_fullpath = 'discharge_target.csv'

# Load the data
refs_df = pd.read_csv(refs_fullpath)
hyps_df = pd.read_csv(hyps_fullpath)

# hyps_df = pd.read_csv(os.path.join(data_dir, "targets.csv"))

# results = pd.DataFrame(columns=["BLEU", "ROUGE_1", "ROUGE_2", "ROUGE_L", "METEOR", "CIDEr", "BERTScore", "BLEURT"])
# refs_df = pd.read_csv()
# hyps_df = pd.read_csv("C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\CODE\discharge_me\data\scoring\submission_01\discharge_target.csv")

# hyps_df.head()
refs_df.head()
print(refs_df.shape)
print(hyps_df.shape)



(10962, 3)
(10962, 6)


In [3]:
## scoring script 

import pandas as pd
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction, corpus_bleu 
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import evaluate
import torch

# nltk.download('punkt')
# Initialize evaluator
device = "cuda" if torch.cuda.is_available() else "cpu"

def load_data(refs_fullpath, hyps_fullpath, cols, data_range):
    # Load the data
    refs_data = pd.read_csv(refs_fullpath)
    hyps_data = pd.read_csv(hyps_fullpath)

    refs_df = refs_data[cols][data_range].replace("\n", " ") ## set range!!!
    hyps_df = hyps_data[cols][data_range].replace("\n", " ")

    return refs_df, hyps_df

def calculate_scores(refs_df, hyps_df):
    refs = refs_df.tolist()
    hyps = hyps_df.tolist()

    # BERTScore Calculation
    P, R, F1 = score(cands=hyps, refs=refs, lang="en", verbose=True)
    
 
    # BLEU and ROUGE scores initialization
    bleu_scores = []
    rouge_scores = []
    rouge = Rouge()


    for ref, hyp in zip(refs, hyps):
        # BLEU Score Calculation

        bleu_score = sentence_bleu([ref.split()], hyp.split(), weights=(0.25, 0.25, 0.25, 0.25))
        # bleu_score = corpus_bleu([ref.split()], hyp.split(), smoothing_function=SmoothingFunction().method1)

        bleu_scores.append(bleu_score)


        # ROUGE Score Calculation
        scores = rouge.get_scores(hyp, ref)[0]
        rouge_scores.append(scores)

        
    # METEOR Score Calculation
    # Initialize the METEOR scorer
    meteor_scorer = evaluate.load('meteor')
    meteor_scores = [meteor_scorer.compute(predictions=[hyp], references=[ref])['meteor'] for ref, hyp in zip(refs, hyps)]

    # Adding scores to dataframe
    results_df = pd.DataFrame({
        # 'discharge_instructions': refs_df['discharge_instructions'],


        'BLEU': bleu_scores,
        'ROUGE-1': [score['rouge-1']['f'] for score in rouge_scores],
        'ROUGE-2': [score['rouge-2']['f'] for score in rouge_scores],
        'ROUGE-L': [score['rouge-l']['f'] for score in rouge_scores],
        'F1 Score (BERT)': F1.numpy(),
        'Precision (BERT)': P.numpy(),
        'Recall (BERT)': R.numpy(),
        'METEOR': meteor_scores
    })
    
    return results_df

# Load the data from discharge instructions
cols = 'discharge_instructions'
data_range = slice(0, 250)
refs_df, hyps_df = load_data(refs_fullpath, hyps_fullpath, cols, data_range)
results = calculate_scores(refs_df, hyps_df)
# print(results)
print('Discharge_instructions:')
mean_scores = results.mean()
print(mean_scores) 
print("overall mean: " + str(mean_scores.mean()))
# selected_metrics = ['F1 Score (BERT)', 'BLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'METEOR']
# print("overall mean: " + str(mean_scores[selected_metrics].mean()))
scores_dishcarge_instructions = results
mean_scores_discharge_instructions = mean_scores
# average the scores for only selected metrics
overall_scores_discharge_instructions = mean_scores.mean()

print(mean_scores_discharge_instructions) 
mean_scores_bhc.to_csv('mean_scores_discharge_instructions.csv')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 8/8 [08:01<00:00, 60.18s/it] 


computing greedy matching.


100%|██████████| 4/4 [00:00<00:00,  4.66it/s]
c:\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


done in 482.32 seconds, 0.52 sentences/sec


c:\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading pack

Discharge_instructions:
BLEU                0.038683
ROUGE-1             0.272309
ROUGE-2             0.078433
ROUGE-L             0.258637
F1 Score (BERT)     0.835737
Precision (BERT)    0.833775
Recall (BERT)       0.838135
METEOR              0.230086
dtype: float64
overall mean: 0.4232243113085784
BLEU                0.038683
ROUGE-1             0.272309
ROUGE-2             0.078433
ROUGE-L             0.258637
F1 Score (BERT)     0.835737
Precision (BERT)    0.833775
Recall (BERT)       0.838135
METEOR              0.230086
dtype: float64
overall mean: 0.4232243113085784


In [4]:

# Load the data from brief_hospital_course
cols = 'brief_hospital_course'
data_range = slice(0, 250)
refs_df, hyps_df = load_data(refs_fullpath, hyps_fullpath, cols, data_range)
results = calculate_scores(refs_df, hyps_df)
mean_scores = results.mean()
print(mean_scores) 
print("overall mean: " + str(mean_scores.mean()))
# selected_metrics = ['F1 Score (BERT)', 'BLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'METEOR']
# print("overall mean: " + str(mean_scores[selected_metrics].mean()))
scores_brief_hospital_course = results
mean_scores_bhc= mean_scores
# average the scores for only selected metrics
overall_scores_brief_hospital_course = mean_scores.mean()

print(mean_scores_bhc) 
mean_scores_bhc.to_csv('mean_scores_bhc.csv')



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 8/8 [08:40<00:00, 65.06s/it]


computing greedy matching.


100%|██████████| 4/4 [00:00<00:00,  4.49it/s]
c:\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


done in 521.39 seconds, 0.48 sentences/sec


c:\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU                0.026207
ROUGE-1             0.229334
ROUGE-2             0.068346
ROUGE-L             0.216008
F1 Score (BERT)     0.821601
Precision (BERT)    0.805791
Recall (BERT)       0.838565
METEOR              0.241789
dtype: float64
overall mean: 0.4059550385421707


In [7]:
print(mean_scores_discharge_instructions) 
mean_scores_bhc.to_csv('mean_scores_discharge_instructions.csv')
print(mean_scores_discharge_instructions) 
mean_scores_bhc.to_csv('mean_scores_discharge_instructions.csv')

BLEU                0.038683
ROUGE-1             0.272309
ROUGE-2             0.078433
ROUGE-L             0.258637
F1 Score (BERT)     0.835737
Precision (BERT)    0.833775
Recall (BERT)       0.838135
METEOR              0.230086
dtype: float64


In [6]:
# print(results)
# mean_scores = results.mean()
print(mean_scores_bhc) 
mean_scores_bhc.to_csv('mean_scores_bhc.csv')
# print("overall mean: " + str(mean_scores.mean()))
# scores_brief_hosptial_course = results
# mean_scores_brief_hospital_course = mean_scores
# selected_metrics = ['F1 Score (BERT)', 'BLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'METEOR']
# overall_scores_brief_hospital_course = mean_scores[selected_metrics].mean()

BLEU                0.026207
ROUGE-1             0.229334
ROUGE-2             0.068346
ROUGE-L             0.216008
F1 Score (BERT)     0.821601
Precision (BERT)    0.805791
Recall (BERT)       0.838565
METEOR              0.241789
dtype: float64


In [114]:
print(mean_scores_discharge_instructions['F1 Score (BERT)'])


0.8430268168449402


In [108]:
print("overall mean discharge_instructions: " + str(overall_scores_brief_hospital_course))
print(mean_scores_discharge_instructions)
print("overall mean brief_hospital_course: " + str(overall_scores_brief_hospital_course))
print(mean_scores_brief_hospital_course)

overall mean discharge_instructions: 0.4053250370443757
Precision (BERT)    0.845265
Recall (BERT)       0.841503
F1 Score (BERT)     0.843027
BLEU                0.038251
ROUGE-1             0.288282
ROUGE-2             0.087102
ROUGE-L             0.273315
METEOR              0.233272
dtype: float64
Precision (BERT)    0.807320
Recall (BERT)       0.831882
F1 Score (BERT)     0.818700
BLEU                0.030160
ROUGE-1             0.239315
ROUGE-2             0.079027
ROUGE-L             0.225743
METEOR              0.210452
dtype: float64


In [120]:
print(mean_scores_discharge_instructions)
print("overall mean discharge instructions: " + str(overall_scores_discharge_instructions))
print(mean_scores_brief_hospital_course)
print("overall mean brief hospital course: " + str(overall_scores_brief_hospital_course))


Precision (BERT)    0.838591
Recall (BERT)       0.838805
F1 Score (BERT)     0.838488
BLEU                0.041435
ROUGE-1             0.275827
ROUGE-2             0.082898
ROUGE-L             0.263232
METEOR              0.224325
dtype: float64
overall mean discharge instructions: 0.28770092599958014
Precision (BERT)    0.804995
Recall (BERT)       0.839190
F1 Score (BERT)     0.821507
BLEU                0.029939
ROUGE-1             0.232463
ROUGE-2             0.073925
ROUGE-L             0.219769
METEOR              0.247535
dtype: float64
overall mean brief hospital course: 0.2708562645384736


In [99]:
scores_brief_hosptial_course.head()
# scores_dishcarge_instructions.head()

,Precision (BERT),Recall (BERT),F1 Score (BERT),BLEU,ROUGE-1,ROUGE-2,ROUGE-L
0,0.802027,0.853258,0.826850,4.774451e-02,0.279070,0.087824,0.250000
1,0.832282,0.830727,0.831504,2.216288e-79,0.207254,0.043636,0.196891
2,0.795514,0.838217,0.816307,9.313128e-02,0.267003,0.131148,0.256927
3,0.846126,0.809325,0.827316,1.572443e-02,0.271605,0.150235,0.259259
4,0.775890,0.845338,0.809126,6.927265e-79,0.218182,0.035443,0.196364


In [ ]:

# Define your reference and hypothesis texts here
refs_text = """
Ms. ___, 
.
You were admitted to the hospital with an infection of the skin 
on the left side of your face.  The infection did not affect 
your eye.  We gave you antibiotics for your infection and it 
improved.  You should continue augmentin by mouth for 3 days.  
.
During your admission, you were also noted to have an elevated 
INR.  Your coumadin was held.  Unfortunately, your INR level 
trended too low, and you were started on a heparin drip.  You 
were discharged on your home coumadin once your level was 
appropriate for your mechanical mitral valve.  
.
Weigh yourself every morning, call MD if weight goes up more 
than 3 lbs.
.
Please follow up with your primary care physician and Dr. ___ 
___ surgeon) as below.  
.
MEDICATIONS CHANGED THIS ADMSSION
START augmentin 875 mg by mouth twice a day for 3 days 
TAKE ___ dose of home coumadin (2.5 mg) on evening of discharge, 
then resume normal coumadin dose

"""

hyps_text = """
Dear patient,

I am writing to inform you about the details of your hospital stay. You were admitted to the hospital because of left-sided facial swelling and pain. You were diagnosed with buccal cellulitis with a possible dental source. Your INR was 7.2, and you were given Unasyn for treatment. 

During your hospital stay, you were prescribed the following medications: 

1. metoprolol tartrate 50 mg Tablet Sig: One (1) Tablet PO BID (2 times a day). 
2. alendronate 70 mg Tablet Sig: One (1) Tablet PO once a week. 
3. polyethylene glycol 3350 17 gram Powder in Packet Sig: One (1) Powder in Packet PO DAILY (Daily) as needed for constipation. 
4. simvastatin 20 mg Tablet Sig: One (1) Tablet PO once a day. 
5. aspirin 81 mg Tablet, Chewable Sig: One (1) Tablet, Chewable PO DAILY (Daily). 
6. Vitamin D3  Oral
7. lisinopril 5 mg Tablet Sig: One (1) Tablet PO once a day. 
8. warfarin 5 mg Tablet Sig: One (1) Tablet PO once a day: take 2.5mg ___ tablet) on evening of discharge, then resume home dose. 
9. amoxicillin-pot clavulanate 875-125 mg Tablet Sig: One (1) Tablet PO Q12H (every 12 hours) for 3.5 days.

You will need to follow up with your primary care physician and dentist as an outpatient to prepare for mouth clearance surgery. You will also need to return to the clinic two days following discharge for an INR check. 

Please follow your medication regimen as prescribed and contact your healthcare provider if you have any concerns or questions. 

Best wishes for a speedy recovery!

"""

# Preprocess texts
refs = [preprocess_text(refs_text)]
hyps = [preprocess_text(hyps_text)]



In [77]:
refs_text = refs_df['discharge_instructions'][1]    

hyps_text = hyps_df['discharge_instructions'][1]

# Preprocess texts
refs = [preprocess_text(refs_text)]
hyps = [preprocess_text(hyps_text)]

In [83]:
refs?

Type:        list
String form: ["Discharge Instructions ---\nDear ,\n\nIt was a pleasure to take care of you during your recent  <...> ase call our office (555-555-5555) or 911 for immediate assistance.\n\nSincerely,\nYour Team---"]
Length:      1
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.

In [84]:
import numpy as np
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import evaluate
import torch



list_of_references = [[ref.split()] for ref in refs]
hypotheses = [hyp.split() for hyp in hyps]
bleu_score = corpus_bleu(list_of_references, hypotheses, smoothing_function=SmoothingFunction().method1)




# Initialize the ROUGE scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
# Initialize the METEOR scorer
meteor_scorer = evaluate.load('meteor')

# Calculate ROUGE scores
rouge_scores = [rouge_scorer.score(ref, hyp) for ref, hyp in zip(refs, hyps)]
# Calculate BERTScores
_, _, bert_scores_f1 = bert_score(refs, hyps, lang="en", verbose=True)
# Calculate METEOR scores
meteor_scores = [meteor_scorer.compute(predictions=[hyp], references=[ref])['meteor'] for ref, hyp in zip(refs, hyps)]
# # Calculate BLEU score
# bleu_score = calculate_bleu(refs, hyps)

# Convert BERTScores' F1 scores from tensors to NumPy arrays and calculate the mean
bert_scores_f1_mean = torch.mean(bert_scores_f1).item()

# Print scores
print("BLEU Score:", bleu_score)
print("ROUGE Scores:")
for rouge_type in ['rouge1', 'rouge2', 'rougeL']:
    mean_score = np.mean([score[rouge_type].fmeasure for score in rouge_scores])
    print(f" {rouge_type}: {mean_score}")
print("BERTScore (F1):", bert_scores_f1_mean)
print("METEOR Score:", np.mean(meteor_scores))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 500.22it/s]

done in 1.20 seconds, 0.83 sentences/sec
BLEU Score: 0.020650183521959388
ROUGE Scores:
 rouge1: 0.45637583892617456
 rouge2: 0.12837837837837837
 rougeL: 0.2550335570469799
BERTScore (F1): 0.8440862894058228
METEOR Score: 0.24374932036102426


In [ ]:
import numpy as np
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import evaluate
import torch

def calculate_bleu(refs, hyps):
    list_of_references = [[ref.split()] for ref in refs]
    hypotheses = [hyp.split() for hyp in hyps]
    bleu_score = corpus_bleu(list_of_references, hypotheses, smoothing_function=SmoothingFunction().method1)
    return bleu_score


# Initialize the ROUGE scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
# Initialize the METEOR scorer
meteor_scorer = evaluate.load('meteor')

# Calculate ROUGE scores
rouge_scores = [rouge_scorer.score(ref, hyp) for ref, hyp in zip(refs, hyps)]
# Calculate BERTScores
_, _, bert_scores_f1 = bert_score(refs, hyps, lang="en", verbose=True)
# Calculate METEOR scores
meteor_scores = [meteor_scorer.compute(predictions=[hyp], references=[ref])['meteor'] for ref, hyp in zip(refs, hyps)]
# Calculate BLEU score
bleu_score = calculate_bleu(refs, hyps)

# Convert BERTScores' F1 scores from tensors to NumPy arrays and calculate the mean
bert_scores_f1_mean = torch.mean(bert_scores_f1).item()

# Print scores
print("BLEU Score:", bleu_score)
print("ROUGE Scores:")
for rouge_type in ['rouge1', 'rouge2', 'rougeL']:
    mean_score = np.mean([score[rouge_type].fmeasure for score in rouge_scores])
    print(f" {rouge_type}: {mean_score}")
print("BERTScore (F1):", bert_scores_f1_mean)
print("METEOR Score:", np.mean(meteor_scores))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\s.wendelken\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
tokenizer_config.json: 100%|██████████| 25.0/25.0 [00:00<?, ?B/s]
C:\Users\s.wendelken\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\s.wendelken\.cache\huggingface\hub\models--roberta-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled b

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:02<00:00,  2.90s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 50.89it/s]


done in 2.92 seconds, 0.34 sentences/sec
BLEU Score: 0.08612427965941073
ROUGE Scores:
 rouge1: 0.3752969121140143
 rouge2: 0.14319809069212408
 rougeL: 0.16627078384798097
BERTScore (F1): 0.8264833092689514
METEOR Score: 0.3243030797985192
